In [1]:
# import all necessary packages for webscraping a list of Bundesliga first devision teams

import time
import requests
from bs4 import BeautifulSoup
import re # support regular expressions
import pandas as pd
import numpy as np

In [2]:
# act as a browser to avoid the hp suspecting any scraping

headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

In [6]:
# get content of the page that lists all current Bundeliga teams on tm.de

page = "https://www.transfermarkt.de/premier-league/startseite/wettbewerb/GB1"
page_tree = requests.get(page, headers=headers)
html_soup = BeautifulSoup(page_tree.content, 'html.parser')

In [7]:
# extracting table from the soup object
club_table = html_soup.find(id='yw1')

In [8]:
club_name_elements = club_table.find_all('td', {'class': 'hauptlink no-border-links'})
club_names = [club.text.replace(u'\xa0', u'').strip() for club in club_name_elements]
club_names

['Manchester City',
 'FC Chelsea',
 'FC Liverpool',
 'FC Arsenal',
 'Manchester United',
 'Tottenham Hotspur',
 'Newcastle United',
 'West Ham United',
 'Leicester City',
 'Aston Villa',
 'Wolverhampton Wanderers',
 'FC Southampton',
 'FC Everton',
 'Nottingham Forest',
 'Crystal Palace',
 'Brighton & Hove Albion',
 'FC Brentford',
 'Leeds United',
 'FC Fulham',
 'AFC Bournemouth']

In [9]:
team_size_elements = club_table.find_all('td', {'class': 'zentriert'})
team_sizes = [size.text for size in team_size_elements][4::4]
team_sizes

['24',
 '32',
 '30',
 '23',
 '32',
 '25',
 '29',
 '22',
 '27',
 '23',
 '28',
 '30',
 '25',
 '36',
 '24',
 '24',
 '24',
 '24',
 '24',
 '27']

In [10]:
avg_team_age_elements = club_table.find_all('td', {'class': 'zentriert'})
avg_team_ages = [age.text for age in avg_team_age_elements][5::4]
avg_team_ages

['26,8',
 '25,3',
 '26,5',
 '25,1',
 '25,7',
 '26,4',
 '27,2',
 '28,1',
 '26,9',
 '27,0',
 '26,0',
 '25,3',
 '27,2',
 '27,7',
 '27,0',
 '24,8',
 '25,5',
 '25,5',
 '28,0',
 '26,3']

In [11]:
avg_team_value_elements = club_table.find_all('td', {'class': 'rechts'})
avg_team_values = [float(value.text.split()[0].replace(',' , '.')) for value in avg_team_value_elements][2::2]
avg_team_values

[43.7,
 32.33,
 31.03,
 34.91,
 23.73,
 28.45,
 16.36,
 21.43,
 16.86,
 19.38,
 14.06,
 12.92,
 14.72,
 9.24,
 12.88,
 12.86,
 12.71,
 12.29,
 10.23,
 8.4]

In [12]:
# finding each teams total value and put in a list
avg_team_value_elements = club_table.find_all('td', {'class': 'rechts'})
team_values = [float(value.text.split()[0].replace(',' , '.')) for value in avg_team_value_elements][3::2]
team_values

[1.05,
 1.03,
 931.0,
 803.0,
 759.2,
 711.3,
 474.4,
 471.5,
 455.1,
 445.8,
 393.7,
 387.5,
 368.1,
 332.65,
 309.2,
 308.6,
 305.1,
 295.0,
 245.5,
 226.7]

In [13]:
# creating a pandas data frame of the tm.de table
df = pd.DataFrame(list(zip(club_names, team_sizes)),columns=['clubs', 'squad'])
df

,clubs,squad
0,Manchester City,24
1,FC Chelsea,32
2,FC Liverpool,30
3,FC Arsenal,23
4,Manchester United,32
5,Tottenham Hotspur,25
6,Newcastle United,29
7,West Ham United,22
8,Leicester City,27
9,Aston Villa,23


In [14]:
df['avg_age'] = pd.Series(avg_team_ages)
df['avg_market_value'] = pd.Series(avg_team_values)
df['market_value'] = pd.Series(team_values)
df

,clubs,squad,avg_age,avg_market_value,market_value
0,Manchester City,24,"26,8",43.70,1.05
1,FC Chelsea,32,"25,3",32.33,1.03
2,FC Liverpool,30,"26,5",31.03,931.00
3,FC Arsenal,23,"25,1",34.91,803.00
4,Manchester United,32,"25,7",23.73,759.20
5,Tottenham Hotspur,25,"26,4",28.45,711.30
6,Newcastle United,29,"27,2",16.36,474.40
7,West Ham United,22,"28,1",21.43,471.50
8,Leicester City,27,"26,9",16.86,455.10
9,Aston Villa,23,"27,0",19.38,445.80
